In [ ]:
import warnings
warnings.filterwarnings("ignore")

import math
import matplotlib.pyplot as plt
import pandas as pd

from collections import namedtuple

%config InlineBackend.figure_format = 'retina'
plt.rc('pdf', fonttype=42) # Avoid type 3 fonts

SEC = 1
MIN = 60 * SEC
HOUR = 60 * MIN
DAY = 24 * HOUR
YEAR = 365 * DAY

In [ ]:
#
# Table: password types and times
#

PasswordConfig = namedtuple('PasswordConfig', ['name', 'elements', 'len'])


ELEMENTS_ALPHANUMERIC = 26+26+10
ELEMENTS_EFF_WORDLIST = 6**5
ELEMENTS_PIN = 10


PASSWORD_CONFIGS = [
    PasswordConfig("AlphaNum (5 chars)", ELEMENTS_ALPHANUMERIC, 5),
    PasswordConfig("AlphaNum (6 chars)", ELEMENTS_ALPHANUMERIC, 6),
    PasswordConfig("AlphaNum (7 chars)", ELEMENTS_ALPHANUMERIC, 7),
    PasswordConfig("WordList (3 words)", ELEMENTS_EFF_WORDLIST, 3),
    PasswordConfig("WordList (4 words)", ELEMENTS_EFF_WORDLIST, 4),
    PasswordConfig("PIN (9 digits)", ELEMENTS_PIN, 9),
]


def entropy(pw_conf):
    return pw_conf.len * math.log(pw_conf.elements, 2)
 

def t_individual(t_total, entropy):
    return t_total / 2**entropy

def tf(tsec):
    
    if tsec < 0.0001:
        return f"<0.1\\,\\si{{ms}}"
    elif tsec < 0.1:
        tms = tsec * 1000
        return f"{tms:.1f}\\,\\si{{ms}}"
    else:
        return f"{tsec:.1f}\\,\\si{{s}}"


for pw_conf in PASSWORD_CONFIGS:
    e = entropy(pw_conf)
    y50 = tf(t_individual(50*YEAR, e))
    y100 = tf(t_individual(100*YEAR, e))
    print(f"{pw_conf.name} & {e:.1f} & {y50} & {y100}\\\\")

In [ ]:
#
# Table: parameter choices Android
#

adf = pd.read_csv('output/android-fitted-durations.csv')

TIME_SMALL_MS = 67
TIME_LARGE_MS = 555


def calc_l(name, t):
    return adf[adf.name==name][adf.duration > t]['size'].min()
    
def fl(x):
    if x >= 1000:
        return f"{x//1000},{x%1000:03}"
    else:
        return x

# output for table
android_device_names = sorted(list(set(adf['name'])))
for name in android_device_names:
    l1 = fl(calc_l(name, TIME_SMALL_MS))
    l2 = fl(calc_l(name, TIME_LARGE_MS))
    name = name.replace(" 5G", "")
    print(f"{name} & {l1} & {l2}\\\\")

# and write to benchmark config file
android_table_file = '../android/testing/src/main/java/com/lambdapioneer/sloth/testing/ConfigTables.kt'
MAP_NAME_TO_DEVICE_NAME = {
    'Google Pixel 3': 'Google Pixel 3',
    'Google Pixel 7': 'Google Pixel 7',
    'Samsung Galaxy S21': 'samsung SM-G991U1',
    'Samsung Galaxy S22 5G': 'samsung SM-S901U1',
}

with open(android_table_file, 'w') as f:
    f.write("// This is a generated file, see `02-calculations-parameters.ipynb`\n")
    f.write(f"\n")
    f.write("package com.lambdapioneer.sloth.generated\n")
    f.write(f"\n")
    for t, constant in zip((TIME_SMALL_MS, TIME_LARGE_MS), ("CONFIG_TABLE_SMALL", "CONFIG_TABLE_LARGE")):
        f.write(f"internal val {constant} = mapOf(\n")
        for name in android_device_names:
            l = calc_l(name, t)
            device_name = MAP_NAME_TO_DEVICE_NAME[name]
            f.write(f"    \"{device_name}\" to {l},\n")
            
        f.write(")\n")
        f.write("\n")

In [ ]:
#
# Table: parameter choices iOS
#

idf = pd.read_csv('output/ios-p10-times.csv')


def calc_n(name, t):
    op_t = idf[idf.Device==name]['Measurement'].mean()
    return math.ceil(t / op_t)
    

for name in ["iPhone 11", "iPhone 12", "iPhone 13"]:
    n1 = calc_n(name, TIME_SMALL_MS)
    n2 = calc_n(name, TIME_LARGE_MS)
    print(f"{name} & {n1} & {n2}\\\\")